<a href="https://colab.research.google.com/github/Yk-rin04/Prog2kakushin/blob/main/work1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Work1
pandasを使ったプログラム

AKB48のCDシングルのデータを用いた，売り上げ枚数ランキング（ランキングの中の欲しい情報だけ抜き出せるようにもする），曲ごとの売り上げ枚数の変化のグラフ，などを表示できるプログラム